In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from scipy import stats
import statsmodels.api as sm

In [31]:
df_excess_return = pd.read_csv("data/q3/excess_returns.csv")
df_excess_return.set_index('date', inplace=True)

# France and Germany are the same
df = df_excess_return.drop(columns = ['X_germany'])

div_df = pd.read_csv("data/q3/div_return.csv")
div_df.set_index('date', inplace=True)
div = div_df["DIV_return"] 

In [32]:
# Q7 a

N = 5 # Number of foreign currency traded

# France and Germany are the same
df = df_excess_return.drop(columns = ['X_germany'])

dollar_ret = - (1/N) * df.sum(axis = 1)

dollar_df = pd.DataFrame({
    "DOLLAR": dollar_ret,
}).dropna()

#df_hedged_returns = pd.read_csv("data/q3/hedged_returns.csv", parse_dates=['date'], index_col='date')
#print("hedged returns \n", df_hedged_returns.head())
#hedged_return_cols = [col for col in df_hedged_returns.columns if col.endswith('_hedged_return_usd')]
# shorting R with R = Sum(Xi / N), and Xi = hedged_return_i
#N = len(hedged_return_cols)
#df_hedged_returns["return"] = - df_hedged_returns[hedged_return_cols].sum(axis=1) / N 
#df_short = df_hedged_returns[['return']].copy()
3#print(df_short.head(5))
#df_short.to_csv("data/q7/USD_vs_World.csv")
# TODO plot evolution of return and verify data on countries (not currency) is correct

3

In [33]:
# Q7 b
def annualized_stats(returns, rf = None, period = 12, zero_cost = False):
    """
    Given two pandas Series of monthly returns (decimal):
      - returns: strategy returns
      - rf:      risk-free returns
    Returns (annualized_mean, annualized_vol, annualized_sharpe).
    """
    mean_annual_return = period * returns.mean()
    std_annual_return = np.sqrt(period) * returns.std()
    if zero_cost:
        return mean_annual_return, std_annual_return, (mean_annual_return/std_annual_return)
    rf_annual = period * rf.mean()
    excess_ret = mean_annual_return - rf_annual
    sharpe_ratio = excess_ret / std_annual_return
    return mean_annual_return, std_annual_return, sharpe_ratio

mean_dollar, sigma_dollar, sharpe_dollar = annualized_stats(dollar_df["DOLLAR"], zero_cost=True)

# One-sample t-test: H0: mean(CARRY) = 0
t_stat_mom, p_val_mom = stats.ttest_1samp(dollar_df["DOLLAR"], popmean=0.0)

print("=== DOLLAR Strategy Statistics ===")
print(f"Long USD vs World Return   : Annualized Return = {mean_dollar:.2%}, "
      f"Volatility = {sigma_dollar:.2%}, Sharpe = {sharpe_dollar:.2f}")

print(f"DOLLAR mean-return t-stat = {t_stat_mom:.3f}, p-value = {p_val_mom:.3f}\n")

alpha = 0.05  # typical significance level
if p_val_mom < alpha:
    print("The strategy's mean return is significantly different from zero.")
else:
    print("The strategy's mean return is not significantly different from zero.")

dollar_df['DOLLAR'].to_csv('data/q7/USD_vs_World.csv')

#def return_stats(returns, periods_per_year=12, risk_free_rate=0):
#    mean = returns.mean()
#    std = returns.std()

#    sharpe = ((mean * periods_per_year) - risk_free_rate) / (std * np.sqrt(periods_per_year))

#    return mean, std, sharpe


#stats = return_stats(df_short['return'])
#print(f"Short USD vs World Return:\n Mean: {stats[0]}, Std: {stats[1]}, Sharpe Ratio: {stats[2]}")

# TODO compare, test if average return is statistically significantly different from 0

=== DOLLAR Strategy Statistics ===
Long USD vs World Return   : Annualized Return = 3.08%, Volatility = 7.74%, Sharpe = 0.40
DOLLAR mean-return t-stat = 1.892, p-value = 0.060

The strategy's mean return is not significantly different from zero.


In [35]:
# Q7 c
# Regress CARRY on DIV (Newey-West errors, lag=1)
reg_mom_div = pd.DataFrame({
    "DOLLAR": dollar_ret.iloc[60:],
    "DIV": div
}).dropna()

X_m = sm.add_constant(reg_mom_div["DIV"])
y_m = reg_mom_div["DOLLAR"]
dollar_on_div = sm.OLS(y_m, X_m).fit()

print(">>> Regression of DOLLAR on DIV:")
print(dollar_on_div.summary())
print("\n")

>>> Regression of DOLLAR on DIV:
                            OLS Regression Results                            
Dep. Variable:                 DOLLAR   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                   0.03112
Date:              mer., 11 juin 2025   Prob (F-statistic):              0.860
Time:                        10:46:58   Log-Likelihood:                 515.63
No. Observations:                 211   AIC:                            -1027.
Df Residuals:                     209   BIC:                            -1021.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.00